# CNN LSTM Univariate

A convolutional neural network, or CNN for short, is a type of neural network developed for
working with two-dimensional image data. The CNN can be very effective at automatically
extracting and learning features from one-dimensional sequence data such as univariate time
series data. A CNN model can be used in a hybrid model with an LSTM backend where the
CNN is used to interpret subsequences of input that together are provided as a sequence to an
LSTM model to interpret. This hybrid model is called a CNN-LSTM.

In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


#### split a univariate sequence into samples

In [2]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

## 1. Data Preparation

#### define input sequence

In [3]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

#### choose a number of time steps

In [4]:
n_steps = 4

#### split into samples

In [5]:
X, y = split_sequence(raw_seq, n_steps)

In [6]:
X

array([[10, 20, 30, 40],
       [20, 30, 40, 50],
       [30, 40, 50, 60],
       [40, 50, 60, 70],
       [50, 60, 70, 80]])

#### reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]

In [7]:
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

In [8]:
X

array([[[[10],
         [20]],

        [[30],
         [40]]],


       [[[20],
         [30]],

        [[40],
         [50]]],


       [[[30],
         [40]],

        [[50],
         [60]]],


       [[[40],
         [50]],

        [[60],
         [70]]],


       [[[50],
         [60]],

        [[70],
         [80]]]])

## 2. CNN LSTM

#### define model

In [9]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

#### fit model

In [10]:
model.fit(X, y, epochs=500, verbose=0)

#### demonstrate prediction

In [11]:
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[102.14299]]
